# Lab 1
exercises in python using minizinc library

---

**How to use special html syntax**
<ul>
    <li><input type="checkbox" checked>Checked element demo.</li>
    <li><input type="checkbox">Unchecked element demo.</li>
</ul>

Highlighted inline formula demo: <span style="background-color: yellow;">*$[X_1, X_2, \ldots, X_n]$*</span>

---

## The N-queens problem
Place n queens in an nxn board so that no 
two queens can attack each other. 
<ul>
    <li><input type="checkbox" checked> Implement the 3 models with GAC on alldifferent.</li>
    <li><input type="checkbox" checked> Search for all solutions for n = 8, 9, 10, 12 using the default search of Gecode.</li>
    <li><input type="checkbox"> Compare the number of solutions and the failures.</li>
</ul>



### 1. Row Model

**Variables and Domains:**

- A variable for each row $[X_1, X_2, ..., X_n]$ → no row attack  
- Domain values $[1..n]$ represent the columns:
  - $X_i = j$ means that the queen in row $i$ is in column $j$

**Constraints:**

- `alldifferent([X_1, X_2, ..., X_n])` → no column attack  
- For all $i < j$, $|X_i - X_j| \ne |i - j|$ → no diagonal attack

In [2]:
import subprocess
import re
from tabulate import tabulate  # pip install tabulate

# Collect results

def compare_results(file_path, values, title, all_solutions=True):
    results = []

    for n in values:

        cmd = [
                "minizinc",
                file_path,
                "-D", f"n={n}",
                "--statistics"
            ]

        if all_solutions:
            cmd.append("--all-solutions")

        # Run MiniZinc with all solutions and statistics
        result = subprocess.run(
            cmd,
            capture_output=True, text=True
        )

        
            

        output = result.stdout

        # ---- Parse output ----
        # Count number of solutions (each ends with "----------")
        num_solutions = output.count("----------")

        # Extract number of failures
        match_failures = re.search(r"%%%mzn-stat: failures=(\d+)", output)
        failures = int(match_failures.group(1)) if match_failures else None

        # Extract solver time
        match_time = re.search(r"%%%mzn-stat: solveTime=([0-9.]+)", output)
        solver_time = float(match_time.group(1)) if match_time else None

        # Add to results list
        results.append([n, num_solutions, failures, solver_time])

    # ---- Print results as a cute table ----
    headers = ["n", "Solutions", "Failures", "Solver Time (s)"]
    table = tabulate(results, headers=headers, floatfmt=".3f", tablefmt="fancy_grid")
    print(f"\n📊 {title}:")
    print(table)

In [7]:
%load_ext iminizinc

<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.9.4, build 2068644735
Copyright (C) 2014-2025 Monash University, NICTA, Data61


In [ ]:
%%writefile "./n-Queens files/queens_row_model.mzn"
include "alldifferent.mzn";

int: n;
array[1..n] of var 1..n: X; % variables & domains 

% column attack constraints
constraint alldifferent(X);

% diagonal attack constraints
constraint forall(i, j in 1..n where i < j)(
    abs(X[i] - X[j]) != abs(i - j)
);

solve satisfy;

Overwriting ./n-Queens files/queens_row_model.mzn


In [13]:
values = [8, 9, 10, 12]
compare_results("./n-Queens files/queens_row_model.mzn", values, "Row Model" )



📊 Row Model:
╒═════╤═════════════╤════════════╤═══════════════════╕
│   n │   Solutions │   Failures │   Solver Time (s) │
╞═════╪═════════════╪════════════╪═══════════════════╡
│   8 │          92 │        921 │             0.009 │
├─────┼─────────────┼────────────┼───────────────────┤
│   9 │         352 │       4655 │             0.047 │
├─────┼─────────────┼────────────┼───────────────────┤
│  10 │         724 │      25125 │             0.251 │
├─────┼─────────────┼────────────┼───────────────────┤
│  12 │       14200 │     773791 │             8.789 │
╘═════╧═════════════╧════════════╧═══════════════════╛


### 2. AllDifferent Model 
**Variables**
- $[X_1, X_2, ..., X_n] \in [1..n]$

**Constraints**
- alldifferent($[X_1, X_2, ..., X_n]$)
- alldifferent($[X_1 + 1,  X_2 + 2, ..., X_n + n]$)
- alldifferent($[X_1 – 1, X_2 – 2, ..., X_n – n]$)


In [ ]:
%%writefile "./n-Queens files/queens_alldiff_model.mzn"
include "alldifferent.mzn";

int: n;
array[1..n] of var 1..n: X; % variables & domains 

% column attack constraints
constraint alldifferent(X);

% diagonal attack constraints
constraint alldifferent([X[i] + i | i in 1..n]);
constraint alldifferent([X[i] - i | i in 1..n]);

solve satisfy;

Overwriting ./n-Queens files/queens_alldiff_model.mzn


In [14]:
values = [8, 9, 10, 12]
compare_results("./n-Queens files/queens_alldiff_model.mzn", values, "AllDifferent Model" )


📊 AllDifferent Model:
╒═════╤═════════════╤════════════╤═══════════════════╕
│   n │   Solutions │   Failures │   Solver Time (s) │
╞═════╪═════════════╪════════════╪═══════════════════╡
│   8 │          92 │        270 │             0.003 │
├─────┼─────────────┼────────────┼───────────────────┤
│   9 │         352 │        965 │             0.013 │
├─────┼─────────────┼────────────┼───────────────────┤
│  10 │         724 │       4221 │             0.047 │
├─────┼─────────────┼────────────┼───────────────────┤
│  12 │       14200 │      88327 │             1.156 │
╘═════╧═════════════╧════════════╧═══════════════════╛


### 3. AllDifferent and Symmetry Breaking Model
**Variables and Domains:**
- for all $i, X_i \in [1..n]$, for all $i, j B_{ij} \in [0..1]$

**Constraints:**
- alldifferent($[X_1, X_2, ..., X_n]$) 
- alldifferent($[X_1 + 1,  X_2 + 2, ..., X_n + n]$)
- alldifferent($[X_1 – 1, X_2 – 2, ..., X_n – n]$) 
- lex≤(B , π(B)) for all π
- _Channeling Constraints_

    - for all $i,j X_i = j B_{ij} = 1 $

[ Section 2.7.6 of the MiniZinc Tutorial ]

In [30]:
%%writefile "./n-Queens files/queens_alldiff_sym_model.mzn"
include "alldifferent.mzn";
include "lex_lesseq.mzn";

int: n; % number of queens
array[1..n] of var 1..n: X; % variables & domains

% column attack constraints
constraint alldifferent(X);

% diagonal attack constraints
constraint alldifferent([X[i] + i | i in 1..n]);
constraint alldifferent([X[i] - i | i in 1..n]);

% dual model
array[1..n, 1..n] of var bool: B; % dual variables & domains

% channeling constraints
constraint forall(i in 1..n, j in 1..n)(
    B[i,j] <-> (X[i]=j)
);

% symmetry breaking constraints
constraint lex_lesseq([B[i,j] | i in 1..n, j in 1..n], [B[j,i] | i in 1..n, j in 1..n]);
constraint lex_lesseq([B[i,j] | i in 1..n, j in 1..n], [B[i,j] | i in reverse(1..n), j in 1..n]);
constraint lex_lesseq([B[i,j] | i in 1..n, j in 1..n], [B[j,i] | i in 1..n, j in reverse(1..n)]);
constraint lex_lesseq([B[i,j] | i in 1..n, j in 1..n], [B[i,j] | i in 1..n, j in reverse(1..n)]);
constraint lex_lesseq([B[i,j] | i in 1..n, j in 1..n], [B[j,i] | i in reverse(1..n), j in 1..n]);
constraint lex_lesseq([B[i,j] | i in 1..n, j in 1..n], [B[i,j] | i in reverse(1..n), j in reverse(1..n)]);
constraint lex_lesseq([B[i,j] | i in 1..n, j in 1..n], [B[j,i] | i in reverse(1..n), j in reverse(1..n)]);

solve satisfy;


Overwriting ./n-Queens files/queens_alldiff_sym_model.mzn


In [35]:
values = [8, 9, 10, 12]
compare_results("./n-Queens files/queens_alldiff_sym_model.mzn", values, "AllDifferent and Symmetry Breaking Model" )


📊 AllDifferent and Symmetry Breaking Model:
╒═════╤═════════════╤════════════╤═══════════════════╕
│   n │   Solutions │   Failures │   Solver Time (s) │
╞═════╪═════════════╪════════════╪═══════════════════╡
│   8 │          12 │         82 │             0.002 │
├─────┼─────────────┼────────────┼───────────────────┤
│   9 │          46 │        309 │             0.013 │
├─────┼─────────────┼────────────┼───────────────────┤
│  10 │          92 │       1025 │             0.034 │
├─────┼─────────────┼────────────┼───────────────────┤
│  12 │        1787 │      20047 │             0.964 │
╘═════╧═════════════╧════════════╧═══════════════════╛


## Result comparison

### Global constraints vs decomposition
- Focus on the alldifferent model.
- Post the alldifferent constraints either by using global constraints or by decomposing them.
- Search for one solution for $N = 28, 29, 30$ using the input order of the variables and the values with Gecode: 
    ``` minizinc
    solve :: int_search(q, input_order, indomain_min) satisfy;
    ```
- Compare the number of failures and the total time. 

In [39]:
n = 8

In [ ]:
%%writefile "./n-Queens files/queens_alldiff_ordered_model.mzn"
include "alldifferent.mzn";

int: n;
array[1..n] of var 1..n: X; % variables & domains 

% column attack constraints
constraint alldifferent(X);

% diagonal attack constraints
constraint alldifferent([X[i] + i | i in 1..n]);
constraint alldifferent([X[i] - i | i in 1..n]);

solve :: int_search(X, input_order, indomain_min) satisfy;

Writing ./n-Queens files/queens_alldiff_ordered_model.mzn


In [3]:
values = [28, 29, 30]
compare_results("./n-Queens files/queens_alldiff_ordered_model.mzn", values, "AllDifferent Model with ordering", False)


📊 AllDifferent Model with ordering:
╒═════╤═════════════╤════════════╤═══════════════════╕
│   n │   Solutions │   Failures │   Solver Time (s) │
╞═════╪═════════════╪════════════╪═══════════════════╡
│  28 │           1 │     146766 │             2.866 │
├─────┼─────────────┼────────────┼───────────────────┤
│  29 │           1 │      68679 │             1.383 │
├─────┼─────────────┼────────────┼───────────────────┤
│  30 │           1 │    2721654 │            52.689 │
╘═════╧═════════════╧════════════╧═══════════════════╛


In [8]:
%%writefile "./n-Queens files/queens_alldiff_dec_model.mzn"
include "alldifferent.mzn";

int: n;
array[1..n] of var 1..n: X; % variables & domains 

% ### AllDiff decomposition ###

% column attack constraint
constraint forall(i, j in 1 .. n where i < j)(X[i] != X[j]);

% diagonal attack constraints
constraint forall(i, j in 1..n where i<j)(X[i]+i != X[j] + j);
constraint forall(i, j in 1..n where i<j)(X[i] - i != X[j] - j);

solve :: int_search(X, input_order, indomain_min) satisfy;

Overwriting ./n-Queens files/queens_alldiff_dec_model.mzn


In [9]:
values = [28, 29, 30]
compare_results("./n-Queens files/queens_alldiff_dec_model.mzn", values, "AllDifferent Model with decomposition", False)


📊 AllDifferent Model with decomposition:
╒═════╤═════════════╤════════════╤═══════════════════╕
│   n │   Solutions │   Failures │   Solver Time (s) │
╞═════╪═════════════╪════════════╪═══════════════════╡
│  28 │           1 │     417027 │             5.682 │
├─────┼─────────────┼────────────┼───────────────────┤
│  29 │           1 │     212257 │             3.054 │
├─────┼─────────────┼────────────┼───────────────────┤
│  30 │           1 │    7472978 │           169.363 │
╘═════╧═════════════╧════════════╧═══════════════════╛


## Results comparison